In [1]:
import sys
import django
import os
file_dir = "/Users/mirbilal/Desktop/MobCommission/commissionV2/"
if file_dir not in sys.path:
    sys.path.insert(0, file_dir)

os.environ["DJANGO_SETTINGS_MODULE"] = "commissionerv2.settings"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true" 
django.setup()
from apps.environment_simulator.models import SimulatedStockBuffer
from apps.environment_simulator.sevice_layer.stock_simulator import StockSimulator
import numpy as np
import json
from datetime import datetime
from dateutil.relativedelta import relativedelta
import pytz
import pandas as pd
import cufflinks as cf

In [8]:
np.random.seed(0)
for i in range(1000):
    starting_latest_time_step = SimulatedStockBuffer.objects.first().captured_at
    # starting_latest_time_step = pytz.utc.localize(datetime.strptime(str(starting_latest_time_step), '%Y-%m-%d %H:%M:%S'))
    starting_next_time_step = starting_latest_time_step + relativedelta(hours= 2, minutes=30)

    latest_time_step = starting_latest_time_step
    next_time_step = starting_next_time_step
    stck_smltr = StockSimulator(latest_time_step, next_time_step)
    stck_smltr.vary_stock_prices()

In [9]:
set = 28.5
x = 28.598611740505373 + set*(-0.0013157644634235062+0.0014610710060191429-0.0034156003454395983+0.002740868399142266)
print(x)
print(28.5/28.583523116499876)

28.583523116499876
0.9970779278621654


In [9]:
cf.go_offline()
cf.set_config_file(world_readable=True, theme="white")

buffer = SimulatedStockBuffer.objects.select_related("stock").order_by("captured_at").all()

price_variations = {}
for snapshot in buffer:
    if snapshot.stock.name not in price_variations:
        price_variations[snapshot.stock.name] = []
    price_variations[snapshot.stock.name].append(snapshot.price_snapshot)

# print(price_variations)
for stck_name in price_variations:
    price_snaps = price_variations[stck_name]
    price_array = np.array(price_snaps)
    max_price = max(price_array)
    normalized_snaps = price_array/max_price
    price_variations[stck_name] = normalized_snaps

    

stock_names = list(price_variations.keys())


# print(price_variations)
df_regret_comparisons = pd.DataFrame(price_variations)
df_regret_comparisons.iplot(
    xTitle='time_steps',
    yTitle='Normalized Prices'
)